## Create a Virtual Environment

NOTE: This assumes you have already created a virtual environment (.venv) in the labs/python folder and have the VS Code Python extension installed.

1. In VS Code, press Ctrl+Shift+P and run Python: Select Interpreter

![Python Select Interpreter](./../../../assets/notebooks-python-kernel1.jpg)

2. Choose Enter interpreter path... and browse to the python inside the parent .venv, e.g.:

![Enter interpreter path...](./../../assets/notebooks-python-kernel2.jpg)

4. Click Find...

![Find...](./../../assets/notebooks-python-kernel3.jpg)

5. Navigate to the appropriate folder for the python executable:

Windows: ..\..\ .venv\Scripts\python.exe

![Windows](./../../assets/notebooks-python-kernel4.jpg)

macOS/Linux: ../../.venv/bin/python

![Ubuntu](./../../assets/notebooks-python-kernel8.jpg)

6. Once selected, click on the Select Kernel menu 

![Select Kernel](./../../assets/notebooks-python-kernel5.jpg)

7. Select Python Environments

![Python Environments](./../../assets/notebooks-python-kernel6.jpg)

8. Select the .venv item in the list

![Python Environments](./../../assets/notebooks-python-kernel7.jpg)


## Verify the Notebook is using the Virtual Environment

In [1]:
import sys

if hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix):
    print(f"[OK] Virtual environment active: {sys.prefix}")
else:
    print("[ERROR] NOT running in virtual environment!")
        

[OK] Virtual environment active: d:\__workshops\github\agent-framework-dev-day\labs\python\.venv


## Verify the Python Version

In [2]:
import sys
    
"""Check Python version"""
version = sys.version_info
print(f"[OK] Python {version.major}.{version.minor}.{version.micro}")

if version.major >= 3 and version.minor >= 13:
    print("[OK] Python version is sufficient (>= 3.13")

[OK] Python 3.13.11
[OK] Python version is sufficient (>= 3.13


## Verify Agent Framework is Installed

In [1]:
import subprocess
try:
    result = subprocess.run(
        ["pip", "show", "agent-framework"],
        capture_output=True,
        text=True,
        check=False
    )
    if result.returncode == 0:
        for line in result.stdout.split('\n'):
            if line.startswith('Version:'):
                version = line.split(':')[1].strip()
                print(f"[OK] agent-framework installed (version {version})")
                
    else:
        print("[ERROR] agent-framework package NOT found")
        print("NOTE: This means virtual environment is not activated!")
        
except Exception as e:
    print(f"[ERROR] Error checking package: {e}")
        

[OK] agent-framework installed (version 1.0.0b260107)


## Test Module Imports

If you are on windows you may need to install the pywin32 module (aka. pywintypes)

In [2]:
%pip install pywin32

Note: you may need to restart the kernel to use updated packages.


In [3]:
try:
    from agent_framework.azure import AzureOpenAIChatClient
    print("[OK] Successfully imported agent_framework.azure module")

except ImportError as e:
    print(f"[ERROR] Cannot import agent_framework: {e}")
    print("NOTE: Virtual environment not activated or package not installed!")

[OK] Successfully imported agent_framework.azure module


## Verify Environment Variables

In [4]:
import os
import dotenv

dotenv.load_dotenv()

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")
tenant_id = os.getenv("AZURE_TENANT_ID")
client_id = os.getenv("AZURE_CLIENT_ID")
client_secret = os.getenv("AZURE_CLIENT_SECRET")
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")

search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")
search_index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")

my_initials = os.getenv("MY_INITIALS")

if project_endpoint:
    print(f"[OK] AZURE_AI_PROJECT_ENDPOINT is configured ({project_endpoint})")
else:
    print("[ERROR] AZURE_AI_PROJECT_ENDPOINT not found")

if model_deployment_name:
    print(f"[OK] AZURE_AI_MODEL_DEPLOYMENT_NAME is configured: {model_deployment_name}")
else:
    print("[ERROR] AZURE_AI_MODEL_DEPLOYMENT_NAME not found")

if tenant_id and client_id and client_secret:
    print(f"[OK] Azure AD credentials are configured (tenant: {tenant_id}, client: {client_id})")
else:
    print("[ERROR] Azure AD credentials not found")

if embedding_deployment:
    print(f"[OK] AZURE_OPENAI_EMBEDDING_DEPLOYMENT is configured: {embedding_deployment}")
else:
    print("[ERROR] AZURE_OPENAI_EMBEDDING_DEPLOYMENT not found")

if search_endpoint and search_api_key and search_index_name:
    print(f"[OK] Azure Cognitive Search is configured (endpoint: {search_endpoint}, index: {search_index_name})")
else:
    print("[ERROR] Azure Cognitive Search configuration not found")

if my_initials:
    print(f"[OK] MY_INITIALS is configured: {my_initials}")
else:
    print("[ERROR] MY_INITIALS not found")

[OK] AZURE_AI_PROJECT_ENDPOINT is configured (https://proj-afp-resource.services.ai.azure.com/api/projects/proj-afp)
[OK] AZURE_AI_MODEL_DEPLOYMENT_NAME is configured: gpt-4.1
[OK] Azure AD credentials are configured (tenant: 5c369887-a4a0-4a67-a8d6-a78e017216fc, client: 29d76a9b-eda9-49ae-9111-9cfab7663d42)
[OK] AZURE_OPENAI_EMBEDDING_DEPLOYMENT is configured: text-embedding-ada-002
[OK] Azure Cognitive Search is configured (endpoint: https://srcjan2026afworkshop.search.windows.net, index: support-tickets-index)
[OK] MY_INITIALS is configured: jah


## Verify Connection to Open AI Endpoint

In [4]:
import os
from azure.identity import DefaultAzureCredential
from agent_framework.azure import AzureAIClient, AzureOpenAIChatClient

try:
    credential = DefaultAzureCredential()
    client = AzureAIClient(project_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], 
                            model_deployment_name=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
                            credential=credential)

    agent = client.create_agent(
        name="JokeAgent",
        instructions="You are good at telling jokes."
    )
    result = await agent.run("Tell me a joke about a pirate.")
    print(result.text)

    print("[OK] AzureAIClient initialized successfully")
    print("[OK] Ready to create agents!")

    print("\n---- Testing AzureOpenAIChatClient ----\n")
    aoai_client = AzureOpenAIChatClient(credential=credential)

    agent2 = aoai_client.create_agent(
        name="DadJokeAgent",
        instructions="You are good at telling jokes."
    )
    result2 = await agent2.run("Tell me a dad joke.")
    print(result2.text)

    print("[OK] AzureOpenAIChatClient initialized successfully")
    print("[OK] Ready to create agents!")

except Exception as e:
    print(f"[ERROR] API connection failed: {e}")
        

Why did the pirate go to school?

To improve his arrrrr-ticulation! 🏴‍☠️😂
[OK] AzureAIClient initialized successfully
[OK] Ready to create agents!

---- Testing AzureOpenAIChatClient ----

Why did the scarecrow win an award? Because he was outstanding in his field!
[OK] AzureOpenAIChatClient initialized successfully
[OK] Ready to create agents!
